In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from collections import defaultdict
import math

# Creating the Graph

Here,

In [2]:
def init_graph(G,node_adj_frame):
    G.add_nodes_from([i for i in range(len(node_adj_frame))])
    labels = {}
    labels = node_adj_frame.columns
    for i in range(len(node_adj_frame)):
        snode = node_adj_frame[labels[0]][i]-1
        if snode == 80:
            continue
        temp = node_adj_frame[labels[2]][i]
        if ',' in str(temp):
            sedge_arr = temp.split(',')
            for j in range(0, len(sedge_arr)):
                k = int(sedge_arr[j])
                G.add_edge(snode, k-1)
        elif np.isnan(temp):
            print("ERROR: Not found in the adjacency excel sheet")
        else:
            G.add_edge(snode, int(temp)-1)
    G.remove_node(80)
    return

In [3]:
def init_graph_attr(G,AdjFile,df,col1, col2, col3):
    node_adj_frame = pd.read_excel(AdjFile)
    node_list = node_adj_frame["KGISTalukN"].tolist()
    node_list.insert(80, "")
    nodeAttr = {}
    init_graph(G,node_adj_frame)
    capability_vector = list(zip(df[col1], df[col2], df[col3]))
    node_attri_dict = dict(zip(df["Taluka"],capability_vector))
    node_attri_dict = dict((k.lower(), v) for k, v in node_attri_dict.items())
    for i in range(len(node_adj_frame)):
        temp = {}
        if i == 80:
            continue
        temp["capabilityvector"] = node_attri_dict[node_list[i].lower()]
        temp["nodeStress"] = 0
        temp["name"] = node_list[i]
        nodeAttr[i] = temp
    nt = {}
    nt["capabilityvector"] = node_attri_dict[node_list[226].lower()]
    nt["nodeStress"] = 0
    nt["name"] = "Hadagali"
    nodeAttr[226] = nt
    nx.set_node_attributes(G, nodeAttr)

In [4]:
G = nx.Graph()
df = pd.read_excel('../input_files/Combined3D.xlsx')
# df = pd.read_excel('Combined3D.xlsx')

In [5]:
dist_taluka_dict = defaultdict(list)
for k, v in zip(df["District_GIS"], df["Taluka"]):
    dist_taluka_dict[k].append(v)

In [6]:
def addList(l1,l2):
    for i in range(len(l1)):
        l1[i] = l1[i] + l2[i]
    return l1
def divList(l1,k):
    for i in range(len(l1)):
        l1[i] = l1[i]/k
    return l1
def l2_normalization(l1,l2):
    k = 0
    for i in range(len(l1)):
        k+= (l1[i] - l2[i])**2
    return math.sqrt(k)

In [7]:
def get_node_stress(G):
    taluka_stress_dict = {}
    for n in G.nodes():
        centroid = [0,0,0]
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            # divide by 3?
            G.nodes[n]["nodeStress"] = l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            taluka_stress_dict[G.nodes[n]["name"].lower()]=G.nodes[n]["nodeStress"]
        except(KeyError):
            pass
    return taluka_stress_dict

In [8]:
def get_node_stability(G):
    taluka_stress_dict = {}
    for n in G.nodes():
        centroid = [0,0,0]
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            # divide by 3?
            G.nodes[n]["nodeStress"] = 1 - l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            taluka_stress_dict[G.nodes[n]["name"].lower()]=G.nodes[n]["nodeStress"]
        except(KeyError):
            pass
    return taluka_stress_dict

In [9]:
df['Taluka'] = df['Taluka'].apply(str.lower)
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized MMR", "Normalized IMR", "Normalized PAW")
initialstress = get_node_stress(G)

In [10]:
df["Initial Stress"] = df["Taluka"].map(initialstress)

In [11]:
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (Kuchha Houses-20%)", "Normalized MMR (HM -20%)", "Normalized PAW (HM-20%)")
KHminus20stress = get_node_stress(G)
df["Stress(KH - 20%)"] = df["Taluka"].map(KHminus20stress)

In [12]:
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (Kuchha Houses-10%)", "Normalized MMR (HM -10%)", "Normalized PAW (HM-10%)")
KHminus10stress = get_node_stress(G)
df["Stress(KH - 10%)"] = df["Taluka"].map(KHminus10stress)

In [13]:
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (Kuchha Houses+10%)", "Normalized MMR (HM +10%)", "Normalized PAW (HM+10%)")
KHplus10stress = get_node_stress(G)
df["Stress(KH + 10%)"] = df["Taluka"].map(KHplus10stress)

In [14]:
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (Kuchha Houses+20%)", "Normalized MMR (HM +20%)", "Normalized PAW (HM+20%)")
KHplus20stress = get_node_stress(G)
df["Stress(KH + 20%)"] = df["Taluka"].map(KHplus20stress)

In [23]:
df

,District_GIS,Taluka,Normalized IMR,Normalized IMR (Kuchha Houses-20%),Normalized IMR (Kuchha Houses-10%),Normalized IMR (Kuchha Houses+10%),Normalized IMR (Kuchha Houses+20%),Normalized MMR,Normalized MMR (HM -20%),Normalized MMR (HM -10%),...,Normalized PAW,Normalized PAW (HM-20%),Normalized PAW (HM-10%),Normalized PAW (HM+10%),Normalized PAW (HM+20%),Initial Stress,Stress(KH - 20%),Stress(KH - 10%),Stress(KH + 10%),Stress(KH + 20%)
0,Kalburgi,afzalpur,0.942582,0.951514,0.948637,0.936137,0.927337,0.799013,0.799037,0.799025,...,0.549513,0.562831,0.555660,0.545146,0.540904,0.136598,0.138432,0.137679,0.135375,0.133979
1,Chikkamagaluru,ajjampura,0.858767,0.861778,0.861713,0.855462,0.849992,0.964878,0.964896,0.964887,...,0.792211,0.799088,0.795410,0.789843,0.787542,0.255164,0.255350,0.255372,0.254919,0.254509
2,Kalburgi,aland,0.891863,0.901185,0.898028,0.885329,0.876570,0.824130,0.824155,0.824143,...,0.703663,0.720667,0.711791,0.696810,0.690153,0.306197,0.313530,0.310137,0.301386,0.296652
3,Dharwad,alnavara,0.591869,0.601318,0.597596,0.585900,0.578465,0.770919,0.770943,0.770931,...,0.613828,0.628075,0.620500,0.608718,0.603754,0.053277,0.050005,0.051566,0.055094,0.056997
4,Hassan,alur,0.624183,0.629077,0.627681,0.620426,0.615106,0.857595,0.857613,0.857604,...,0.533341,0.537543,0.534955,0.533450,0.533557,0.189681,0.188180,0.189126,0.190158,0.190341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,Kalburgi,yadrami,0.651312,0.661632,0.657575,0.644782,0.636640,0.827062,0.827087,0.827074,...,0.291834,0.301356,0.295842,0.290482,0.289168,0.292110,0.285553,0.289080,0.294742,0.297097
222,Chamarajanagara,yalandur,0.807541,0.805643,0.807940,0.806799,0.804005,0.868484,0.868492,0.868488,...,0.609085,0.602836,0.605598,0.613894,0.618565,0.403218,0.401504,0.403026,0.403238,0.402247
223,Bengaluru_Urban,yelahanka,0.533541,0.522481,0.528888,0.537960,0.540987,0.992303,0.992299,0.992301,...,0.796274,0.775650,0.785852,0.807167,0.817750,0.317864,0.320499,0.319626,0.315956,0.313429
224,Koppal,yelburga,0.724813,0.732669,0.729964,0.719363,0.712106,0.861132,0.861155,0.861144,...,0.384291,0.391376,0.387185,0.383692,0.383109,0.223837,0.223988,0.224111,0.223477,0.222864


In [24]:
df.to_excel('../output_files_3d/3D_talukaLevel_stress_impact.xlsx')

In [16]:
def aggregate(taluka_stress_dict):
    dist_stress = {}
    for dist, taluks in dist_taluka_dict.items():
        agg_stress = 0
        for taluk in taluks:
            try:
                agg_stress = agg_stress + taluka_stress_dict[taluk.lower()]
            except(KeyError):
                pass
        dist_stress[dist] = agg_stress/len(taluks)
    return dist_stress

In [17]:
df_imp = pd.read_excel('../input_files/3d_Impact_scaled.xlsx')

In [18]:
combined_Impact_HM = pd.DataFrame()
aggregate_df = pd.DataFrame()

In [19]:
# ===================================================FOR HM -20%==============================================================
# Getting the graph ready for HM -20%
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (Kuchha Houses-20%)", "Normalized MMR (HM -20%)", "Normalized PAW (HM-20%)")

# These dict have the names vs Impact Score for HM -20%
IMR_impact_dict = dict(zip(df_imp["Taluka"],df_imp["IMR HI -20% | Impact"]))
MMR_impact_dict = dict(zip(df_imp["Taluka"],df_imp["MMR HI -20% | Impact"]))
PAW_impact_dict = dict(zip(df_imp["Taluka"],df_imp["PAW HI -20% | Impact"]))

# Here, we are just converting the taluka names in these dictionaries to lower case
IMR_impact_dict= dict((k.lower(), v) for k, v in IMR_impact_dict.items())
MMR_impact_dict= dict((k.lower(), v) for k, v in MMR_impact_dict.items())
PAW_impact_dict= dict((k.lower(), v) for k, v in PAW_impact_dict.items())

# Aggregating both the impacts
aggregate_IMR_Impact = aggregate(IMR_impact_dict)
aggregate_MMR_Impact = aggregate(MMR_impact_dict)
aggregate_PAW_Impact = aggregate(PAW_impact_dict)

# Aggregating the stress
aggregate_Stress = aggregate(get_node_stress(G))

# Putting it in a temp df
temp2_df = pd.DataFrame.from_dict([aggregate_IMR_Impact, aggregate_MMR_Impact, aggregate_PAW_Impact, aggregate_Stress])
aI_df = temp2_df.T
aI_df = temp2_df.transpose()
aI_df.rename(columns = {0:'IMPACT_SCORE_IMR (HM - 20%)', 1:'IMPACT_SCORE_MMR (HM - 20%)', 2:'IMPACT_SCORE_PAW (HM - 20%)', 3:'STRESS_SCORE (HM -20%)'}, inplace = True)
aggregate_df = aI_df

# Now, we are converting them into a dataframe and making them Taluka, Impact, Stress
combined_IMR = pd.DataFrame.from_dict([IMR_impact_dict, get_node_stress(G)])
combined_MMR = pd.DataFrame.from_dict([MMR_impact_dict, get_node_stress(G)])
combined_PAW = pd.DataFrame.from_dict([PAW_impact_dict, get_node_stress(G)])

# Now, we are taking a transpose, so that we get it in column form
trdIMR = combined_IMR.T
trdMMR = combined_MMR.T
trdPAW = combined_PAW.T
trdIMR = combined_IMR.transpose()
trdMMR = combined_MMR.transpose()
trdPAW = combined_PAW.transpose()

# Renaming the columns
trdIMR.rename(columns = {0:'IMPACT_SCORE_IMR (HM - 20%)', 1:'STRESS_SCORE (HM - 20%)'}, inplace = True)
trdMMR.rename(columns = {0:'IMPACT_SCORE_MMR (HM - 20%)', 1:'STRESS_SCORE (HM - 20%)'}, inplace = True)
trdPAW.rename(columns = {0:'IMPACT_SCORE_PAW (HM - 20%)', 1:'STRESS_SCORE (HM - 20%)'}, inplace = True)

# Filling the dataframe
combined_Impact_HM = trdIMR
combined_Impact_HM['IMPACT_SCORE_MMR (HM - 20%)'] = trdMMR['IMPACT_SCORE_MMR (HM - 20%)']
combined_Impact_HM['IMPACT_SCORE_PAW (HM - 20%)'] = trdPAW['IMPACT_SCORE_PAW (HM - 20%)']

In [20]:
# ===================================================FOR HM -10%==============================================================
# Getting the graph ready for HM -10%
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (Kuchha Houses-10%)", "Normalized MMR (HM -10%)", "Normalized PAW (HM-10%)")

# These dict have the names vs Impact Score for HM -10%
IMR_impact_dict = dict(zip(df_imp["Taluka"],df_imp["IMR HI -10% | Impact"]))
MMR_impact_dict = dict(zip(df_imp["Taluka"],df_imp["MMR HI -10% | Impact"]))
PAW_impact_dict = dict(zip(df_imp["Taluka"],df_imp["PAW HI -10% | Impact"]))

# Here, we are just converting the taluka names in these dictionaries to lower case
IMR_impact_dict= dict((k.lower(), v) for k, v in IMR_impact_dict.items())
MMR_impact_dict= dict((k.lower(), v) for k, v in MMR_impact_dict.items())
PAW_impact_dict= dict((k.lower(), v) for k, v in PAW_impact_dict.items())

# Aggregating both the impacts
aggregate_IMR_Impact = aggregate(IMR_impact_dict)
aggregate_MMR_Impact = aggregate(MMR_impact_dict)
aggregate_PAW_Impact = aggregate(PAW_impact_dict)

# Aggregating the stress
aggregate_Stress = aggregate(get_node_stress(G))

# Putting it in a temp df
temp2_df = pd.DataFrame.from_dict([aggregate_IMR_Impact, aggregate_MMR_Impact, aggregate_PAW_Impact, aggregate_Stress])
aI_df = temp2_df.T
aI_df = temp2_df.transpose()
aI_df.rename(columns = {0:'IMPACT_SCORE_IMR (HM - 10%)', 1:'IMPACT_SCORE_MMR (HM - 10%)', 2:'IMPACT_SCORE_PAW (HM - 10%)', 3:'STRESS_SCORE (HM -10%)'}, inplace = True)
aggregate_df['IMPACT_SCORE_IMR (HM - 10%)'] = aI_df['IMPACT_SCORE_IMR (HM - 10%)']
aggregate_df['IMPACT_SCORE_MMR (HM - 10%)'] = aI_df['IMPACT_SCORE_MMR (HM - 10%)']
aggregate_df['IMPACT_SCORE_PAW (HM - 10%)'] = aI_df['IMPACT_SCORE_PAW (HM - 10%)']
aggregate_df['STRESS_SCORE (HM - 10%)'] = aI_df['STRESS_SCORE (HM -10%)']

# Now, we are converting them into a dataframe and making them Taluka, Impact, Stress
combined_IMR = pd.DataFrame.from_dict([IMR_impact_dict, get_node_stress(G)])
combined_MMR = pd.DataFrame.from_dict([MMR_impact_dict, get_node_stress(G)])
combined_PAW = pd.DataFrame.from_dict([PAW_impact_dict, get_node_stress(G)])

# Now, we are taking a transpose, so that we get it in column form
trdIMR = combined_IMR.T
trdMMR = combined_MMR.T
trdPAW = combined_PAW.T
trdIMR = combined_IMR.transpose()
trdMMR = combined_MMR.transpose()
trdPAW = combined_PAW.transpose()

# Renaming the columns
trdIMR.rename(columns = {0:'IMPACT_SCORE_IMR (HM - 10%)', 1:'STRESS_SCORE (HM -10%)'}, inplace = True)
trdMMR.rename(columns = {0:'IMPACT_SCORE_MMR (HM - 10%)', 1:'STRESS_SCORE (HM -10%)'}, inplace = True)
trdPAW.rename(columns = {0:'IMPACT_SCORE_PAW (HM - 10%)', 1:'STRESS_SCORE (HM -10%)'}, inplace = True)

# Filling the dataframe
combined_Impact_HM['STRESS_SCORE (HM - 10%)'] = trdPAW['STRESS_SCORE (HM -10%)']
combined_Impact_HM['IMPACT_SCORE_IMR (HM - 10%)'] = trdIMR['IMPACT_SCORE_IMR (HM - 10%)']
combined_Impact_HM['IMPACT_SCORE_MMR (HM - 10%)'] = trdMMR['IMPACT_SCORE_MMR (HM - 10%)']
combined_Impact_HM['IMPACT_SCORE_PAW (HM - 10%)'] = trdPAW['IMPACT_SCORE_PAW (HM - 10%)']

In [21]:
# ===================================================FOR HM +10%==============================================================
# Getting the graph ready for HM +10%
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (Kuchha Houses+10%)", "Normalized MMR (HM +10%)", "Normalized PAW (HM+10%)")

# These dict have the names vs Impact Score for HM +10%
IMR_impact_dict = dict(zip(df_imp["Taluka"],df_imp["IMR HI +10% | Impact"]))
MMR_impact_dict = dict(zip(df_imp["Taluka"],df_imp["MMR HI +10% | Impact"]))
PAW_impact_dict = dict(zip(df_imp["Taluka"],df_imp["PAW HI +10% | Impact"]))

# Here, we are just converting the taluka names in these dictionaries to lower case
IMR_impact_dict= dict((k.lower(), v) for k, v in IMR_impact_dict.items())
MMR_impact_dict= dict((k.lower(), v) for k, v in MMR_impact_dict.items())
PAW_impact_dict= dict((k.lower(), v) for k, v in PAW_impact_dict.items())

# Aggregating both the impacts
aggregate_IMR_Impact = aggregate(IMR_impact_dict)
aggregate_MMR_Impact = aggregate(MMR_impact_dict)
aggregate_PAW_Impact = aggregate(PAW_impact_dict)

# Aggregating the stress
aggregate_Stress = aggregate(get_node_stress(G))

# Putting it in a temp df
temp2_df = pd.DataFrame.from_dict([aggregate_IMR_Impact, aggregate_MMR_Impact, aggregate_PAW_Impact, aggregate_Stress])
aI_df = temp2_df.T
aI_df = temp2_df.transpose()
aI_df.rename(columns = {0:'IMPACT_SCORE_IMR (HM + 10%)', 1:'IMPACT_SCORE_MMR (HM + 10%)', 2:'IMPACT_SCORE_PAW (HM + 10%)', 3:'STRESS_SCORE (HM +10%)'}, inplace = True)
aggregate_df['IMPACT_SCORE_IMR (HM + 10%)'] = aI_df['IMPACT_SCORE_IMR (HM + 10%)']
aggregate_df['IMPACT_SCORE_MMR (HM + 10%)'] = aI_df['IMPACT_SCORE_MMR (HM + 10%)']
aggregate_df['IMPACT_SCORE_PAW (HM + 10%)'] = aI_df['IMPACT_SCORE_PAW (HM + 10%)']
aggregate_df['STRESS_SCORE (HM + 10%)'] = aI_df['STRESS_SCORE (HM +10%)']

# Now, we are converting them into a dataframe and making them Taluka, Impact, Stress
combined_IMR = pd.DataFrame.from_dict([IMR_impact_dict, get_node_stress(G)])
combined_MMR = pd.DataFrame.from_dict([MMR_impact_dict, get_node_stress(G)])
combined_PAW = pd.DataFrame.from_dict([PAW_impact_dict, get_node_stress(G)])

# Now, we are taking a transpose, so that we get it in column form
trdIMR = combined_IMR.T
trdMMR = combined_MMR.T
trdPAW = combined_PAW.T
trdIMR = combined_IMR.transpose()
trdMMR = combined_MMR.transpose()
trdPAW = combined_PAW.transpose()

# Renaming the columns
trdIMR.rename(columns = {0:'IMPACT_SCORE_IMR (HM + 10%)', 1:'STRESS_SCORE (HM + 10%)'}, inplace = True)
trdMMR.rename(columns = {0:'IMPACT_SCORE_MMR (HM + 10%)', 1:'STRESS_SCORE (HM + 10%)'}, inplace = True)
trdPAW.rename(columns = {0:'IMPACT_SCORE_PAW (HM + 10%)', 1:'STRESS_SCORE (HM + 10%)'}, inplace = True)

# Filling the dataframe
combined_Impact_HM['STRESS_SCORE (HM + 10%)'] = trdPAW['STRESS_SCORE (HM + 10%)']
combined_Impact_HM['IMPACT_SCORE_IMR (HM + 10%)'] = trdIMR['IMPACT_SCORE_IMR (HM + 10%)']
combined_Impact_HM['IMPACT_SCORE_MMR (HM + 10%)'] = trdMMR['IMPACT_SCORE_MMR (HM + 10%)']
combined_Impact_HM['IMPACT_SCORE_PAW (HM + 10%)'] = trdPAW['IMPACT_SCORE_PAW (HM + 10%)']

In [22]:
# ===================================================FOR HM +20%==============================================================
# Getting the graph ready for HM +20%
init_graph_attr(G, '../input_files/IMR_Stress_AdjFile.xlsx', df, "Normalized IMR (Kuchha Houses+20%)", "Normalized MMR (HM +20%)", "Normalized PAW (HM+20%)")

# These dict have the names vs Impact Score for HM +20%
IMR_impact_dict = dict(zip(df_imp["Taluka"],df_imp["IMR HI +20% | Impact"]))
MMR_impact_dict = dict(zip(df_imp["Taluka"],df_imp["MMR HI +20% | Impact"]))
PAW_impact_dict = dict(zip(df_imp["Taluka"],df_imp["PAW HI +20% | Impact"]))
MMR_impact_dict= dict((k.lower(), v) for k, v in MMR_impact_dict.items())
PAW_impact_dict= dict((k.lower(), v) for k, v in PAW_impact_dict.items())

# Here, we are just converting the taluka names in these dictionaries to lower case
IMR_impact_dict= dict((k.lower(), v) for k, v in IMR_impact_dict.items())
MMR_impact_dict= dict((k.lower(), v) for k, v in MMR_impact_dict.items())
PAW_impact_dict= dict((k.lower(), v) for k, v in PAW_impact_dict.items())

# Aggregating both the impacts
aggregate_IMR_Impact = aggregate(IMR_impact_dict)
aggregate_MMR_Impact = aggregate(MMR_impact_dict)
aggregate_PAW_Impact = aggregate(PAW_impact_dict)

# Aggregating the stress
aggregate_Stress = aggregate(get_node_stress(G))

# Putting it in a temp df
temp2_df = pd.DataFrame.from_dict([aggregate_IMR_Impact, aggregate_MMR_Impact, aggregate_PAW_Impact, aggregate_Stress])
aI_df = temp2_df.T
aI_df = temp2_df.transpose()
aI_df.rename(columns = {0:'IMPACT_SCORE_IMR (HM + 20%)', 1:'IMPACT_SCORE_MMR (HM + 20%)', 2:'IMPACT_SCORE_PAW (HM + 20%)', 3:'STRESS_SCORE (HM +20%)'}, inplace = True)
aggregate_df['IMPACT_SCORE_IMR (HM + 20%)'] = aI_df['IMPACT_SCORE_IMR (HM + 20%)']
aggregate_df['IMPACT_SCORE_MMR (HM + 20%)'] = aI_df['IMPACT_SCORE_MMR (HM + 20%)']
aggregate_df['IMPACT_SCORE_PAW (HM + 20%)'] = aI_df['IMPACT_SCORE_PAW (HM + 20%)']
aggregate_df['STRESS_SCORE (HM + 20%)'] = aI_df['STRESS_SCORE (HM +20%)']

# Now, we are converting them into a dataframe and making them Taluka, Impact, Stress
combined_IMR = pd.DataFrame.from_dict([IMR_impact_dict, get_node_stress(G)])
combined_MMR = pd.DataFrame.from_dict([MMR_impact_dict, get_node_stress(G)])
combined_PAW = pd.DataFrame.from_dict([PAW_impact_dict, get_node_stress(G)])

# Now, we are taking a transpose, so that we get it in column form
trdIMR = combined_IMR.T
trdMMR = combined_MMR.T
trdPAW = combined_PAW.T
trdIMR = combined_IMR.transpose()
trdMMR = combined_MMR.transpose()
trdPAW = combined_PAW.transpose()

# Renaming the columns
trdIMR.rename(columns = {0:'IMPACT_SCORE_IMR (HM + 20%)', 1:'STRESS_SCORE (HM + 20%)'}, inplace = True)
trdMMR.rename(columns = {0:'IMPACT_SCORE_MMR (HM + 20%)', 1:'STRESS_SCORE (HM + 20%)'}, inplace = True)
trdPAW.rename(columns = {0:'IMPACT_SCORE_PAW (HM + 20%)', 1:'STRESS_SCORE (HM + 20%)'}, inplace = True)

# Filling the dataframe
combined_Impact_HM['STRESS_SCORE (HM + 20%)'] = trdPAW['STRESS_SCORE (HM + 20%)']
combined_Impact_HM['IMPACT_SCORE_IMR (HM + 20%)'] = trdIMR['IMPACT_SCORE_IMR (HM + 20%)']
combined_Impact_HM['IMPACT_SCORE_MMR (HM + 20%)'] = trdMMR['IMPACT_SCORE_MMR (HM + 20%)']
combined_Impact_HM['IMPACT_SCORE_PAW (HM + 20%)'] = trdPAW['IMPACT_SCORE_PAW (HM + 20%)']

In [25]:
combined_Impact_HM

,IMPACT_SCORE_IMR (HM - 20%),STRESS_SCORE (HM - 20%),IMPACT_SCORE_MMR (HM - 20%),IMPACT_SCORE_PAW (HM - 20%),STRESS_SCORE (HM - 10%),IMPACT_SCORE_IMR (HM - 10%),IMPACT_SCORE_MMR (HM - 10%),IMPACT_SCORE_PAW (HM - 10%),STRESS_SCORE (HM + 10%),IMPACT_SCORE_IMR (HM + 10%),IMPACT_SCORE_MMR (HM + 10%),IMPACT_SCORE_PAW (HM + 10%),STRESS_SCORE (HM + 20%),IMPACT_SCORE_IMR (HM + 20%),IMPACT_SCORE_MMR (HM + 20%),IMPACT_SCORE_PAW (HM + 20%)
afzalpur,0.804013,0.138432,0.003543,0.450724,0.137679,0.804013,0.003543,0.463367,0.135375,0.011815,0.000052,0.483767,0.133979,0.023081,0.000102,0.492481
ajjampura,0.599065,0.255350,0.002640,0.220156,0.255372,0.599065,0.002640,0.231825,0.254919,0.008804,0.000039,0.252047,0.254509,0.017197,0.000076,0.261065
aland,0.813601,0.313530,0.003586,0.296689,0.310137,0.813601,0.003586,0.312582,0.301386,0.011956,0.000053,0.340146,0.296652,0.023356,0.000103,0.352443
alnavara,0.797075,0.050005,0.003513,0.387051,0.051566,0.797075,0.003513,0.400746,0.055094,0.011714,0.000052,0.423566,0.056997,0.022881,0.000101,0.433510
alur,0.646132,0.188180,0.002848,0.475402,0.189126,0.646132,0.002848,0.483362,0.190158,0.009495,0.000042,0.494842,0.190341,0.018548,0.000082,0.499374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yadrami,0.830474,0.285553,0.003660,0.705902,0.289080,0.830474,0.003660,0.714287,0.294742,0.012204,0.000054,0.724924,0.297097,0.023840,0.000105,0.728679
yalandur,0.430486,0.401504,0.001897,0.411682,0.403026,0.430486,0.001897,0.415139,0.403238,0.006326,0.000028,0.418665,0.402247,0.012358,0.000054,0.419613
yelahanka,0.103346,0.320499,0.000455,0.243030,0.319626,0.103346,0.000455,0.241056,0.315956,0.001519,0.000007,0.235642,0.313429,0.002967,0.000013,0.232722
yelburga,0.752716,0.223988,0.003317,0.618049,0.224111,0.752716,0.003317,0.626073,0.223477,0.011062,0.000049,0.636658,0.222864,0.021608,0.000095,0.640536


In [26]:
combined_Impact_HM.to_excel('../output_files_3d/3D_talukaLevel_stress_impact.xlsx')

In [27]:
aggregate_df.to_excel('../output_files_3d/3D_districtLevel_stress_impact.xlsx')

In [25]:
# =================================================================SCORE====================================================================================================

In [39]:
df_score = pd.DataFrame()
df_score['Taluka'] = df['Taluka']

In [40]:
def taluka_SI_score(df, score_list, i):
    str_imr = 'IMPACT_SCORE_IMR (HM '+ i + '0%)'
    str_mmr = 'IMPACT_SCORE_IMR (HM '+ i + '0%)'
    str_paw = 'IMPACT_SCORE_PAW (HM '+ i + '0%)'
    str_stress = 'STRESS_SCORE (HM ' + i + '0%)'
    for j in range(len(df)):
        mean_impact = (df[str_imr][j] + df[str_mmr][j] + df[str_paw][j]) / 3
        dissonance = abs(max(df[str_imr][j], df[str_mmr][j], df[str_paw][j]) - min(df[str_imr][j], df[str_mmr][j], df[str_paw][j]))
        score = (mean_impact * (1 - df[str_stress][j])) / dissonance
        score_list.append(score)

In [41]:
def district_SI_score(df, score_list, i, diss_list, imr_impact, mmr_impact, paw_impact, avg_impact):
    str_imr = 'IMPACT_SCORE_IMR (HM '+ i + '0%)'
    str_mmr = 'IMPACT_SCORE_IMR (HM '+ i + '0%)'
    str_paw = 'IMPACT_SCORE_PAW (HM '+ i + '0%)'
    str_stress = 'STRESS_SCORE (HM ' + i + '0%)'
    for j in range(len(df)):
        imr_impact.append(df[str_imr][j])
        mmr_impact.append(df[str_mmr][j])
        paw_impact.append(df[str_paw][j])
        mean_impact = (df[str_imr][j] + df[str_mmr][j] + df[str_paw][j]) / 3 #mean impact of the taluka
        avg_impact.append(mean_impact)
        dissonance = abs(max(df[str_imr][j], df[str_mmr][j], df[str_paw][j]) - min(df[str_imr][j], df[str_mmr][j], df[str_paw][j])) #dissonance between the taluka
        diss_list.append(dissonance)
        score = (mean_impact * (1 - df[str_stress][j])) / dissonance
        score_list.append(score)

In [46]:
def normalize(df, col_str, nd_list, lower_lim, upper_lim):
    for j in range(len(df)):
        norm_dis = 0
        norm_dis = lower_lim + (upper_lim - lower_lim)*(df[col_str][j] - min(df[col_str]))/(max(df[col_str]) - min(df[col_str]))
        nd_list.append(norm_dis)

In [42]:
temp = ['+ 2', '+ 1', '- 1', '- 2']
for i in temp:
    score_list = []
    taluka_SI_score(combined_Impact_HM, score_list, i)
    str = 'SI_Score (HM ' + i + '0%)'
    df_score[str] = score_list

In [48]:
aggregate_df.rename(columns={'STRESS_SCORE (HM -20%)':'STRESS_SCORE (HM - 20%)'}, inplace=True)

In [44]:
df_score.to_excel('../output_files_3d/3D_talukaLevel_SI_score.xlsx')

In [54]:
aggregated_score_df = pd.DataFrame()
aggregated_score_df['District'] = dist_taluka_dict.keys()

In [55]:
for i in temp:
    score_list = []
    diss_list = []
    imr_impact = []
    mmr_impact = []
    paw_impact = []
    avg_impact = []
    district_SI_score(aggregate_df, score_list, i, diss_list, imr_impact, mmr_impact, paw_impact, avg_impact)
    score_str = 'SI_Score (HM ' + i + '0%)'
    str_imr = 'IMPACT_SCORE_IMR (HM '+ i + '0%)'
    str_mmr = 'IMPACT_SCORE_IMR (HM '+ i + '0%)'
    str_paw = 'IMPACT_SCORE_PAW (HM '+ i + '0%)'
    str_stress = 'STRESS_SCORE (HM ' + i + '0%)'
    str_diss = 'DISSONANCE (HM ' + i + '0%)'
    str_avg_imr = 'MEAN IMR (HM ' + i + '0%)'
    aggregated_score_df[score_str] = score_list
    aggregated_score_df[str_imr] = imr_impact
    aggregated_score_df[str_mmr] = mmr_impact
    aggregated_score_df[str_paw] = paw_impact
    aggregated_score_df[str_avg_imr] = avg_impact
    aggregated_score_df[str_diss] = diss_list

In [50]:
# for i in temp:
#     temp_dict = dict(zip(df_score['Taluka'],df_score['SI_Score (HM ' + i + '0%)']))
#     agg_dict = aggregate(temp_dict)
#     aggregated_score_df['SI_Score (HM ' + i + '0%)'] = agg_dict.values()

In [57]:
aggregated_score_df.to_excel('../output_files_3d/3D_districtLevel_SI_score.xlsx')

In [58]:
for i in temp:
    normalized_score_list = []
    score_str = 'SI_Score (HM ' + i + '0%)'
    normalize(aggregated_score_df, score_str, normalized_score_list, 0, 1)
    aggregated_score_df[score_str] = normalized_score_list

In [60]:
aggregated_score_df.to_excel('../output_files_3d/3D_districtLevel_normalizaed_SI_score.xlsx')